Gauss

In [44]:
import numpy as np

y_12 = 1 / (0.01 + 0.01j)
y_13 = 1 / (0.02 + 0.02j)
y_23 = 1 / (0.03 + 0.03j)


# polar form to rectangular
def P2R(A, phi):
    return A * (np.cos(phi) + np.sin(phi) * 1j)


# rectangular to polar
def R2P(x):
    return abs(x), np.angle(x)


Y = np.matrix(
    [
        [y_12 + y_13, -y_12, -y_13],
        [-y_12, y_12 + y_23, -y_23],
        [-y_13, -y_23, y_13 + y_23],
    ]
)

# complex powers at buses 1 and 2
S = [-0.4 + 0.2j, -0.3 + 0.3j]


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


if not check_symmetric(Y):
    raise Exception("not symmetric, something is wrong with your admittance matrix")


# buses 1 and 2
pq_buses = [0, 1]

all_buses = [0, 1, 2]


# voltage and angle, flat start
V = [[(1, 0), (1, 0)]]

i = 0


TOLERANCE = 10e-6


def has_converged(voltage_list):
    cur = voltage_list[-1]
    prev = (
        voltage_list[-2] if len(voltage_list) > 1 else [(0, 0), (0, 0)]
    )  # dummy value if starting out

    for k in pq_buses:
        if np.abs(cur[k][0] - prev[k][0]) > TOLERANCE:
            return False

    return True


while not has_converged(V) and i < 1000:
    i += 1
    new_v = [(0, 0), (0, 0)]  # initialize new voltages with dummy values
    for k in pq_buses:
        sum_term = 0
        for j in all_buses:
            if k != j:
                # slack bus when j == 2
                V_j = P2R(1, 0) if j == 2 else P2R(*V[i - 1][j])
                sum_term += Y[k, j] * V_j

        new_v[k] = (1 / Y[k, k]) * ((np.conjugate(S[k]) / P2R(*V[i - 1][k])) - sum_term)

    new_v_polar = [R2P(vv) for vv in new_v]
    V.append(new_v_polar)


print(f"took {i} iterations")
V[-1]


took 15 iterations


[(0.9976313146636302, -0.014018557035269214),
 (0.9983353371644513, -0.014997561323291783)]

Gauss Seidel

In [46]:
import numpy as np

y_12 = 1 / (0.01 + 0.01j)
y_13 = 1 / (0.02 + 0.02j)
y_23 = 1 / (0.03 + 0.03j)


# polar form to rectangular
def P2R(A, phi):
    return A * (np.cos(phi) + np.sin(phi) * 1j)


# rectangular to polar
def R2P(x):
    return abs(x), np.angle(x)


Y = np.matrix(
    [
        [y_12 + y_13, -y_12, -y_13],
        [-y_12, y_12 + y_23, -y_23],
        [-y_13, -y_23, y_13 + y_23],
    ]
)

# complex powers at buses 1 and 2
S = [-0.4 + 0.2j, -0.3 + 0.3j]


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


if not check_symmetric(Y):
    raise Exception("not symmetric, something is wrong with your admittance matrix")


# buses 1 and 2
pq_buses = [0, 1]

all_buses = [0, 1, 2]


# voltage and angle, flat start
# IMPORTANT! I changed how I am storing the voltage list compared to Gauss method.
# Before my V looked like this:  V = [[(1, 0), (1, 0)]]
V = ([(1, 0)], [(1, 0)])

i = 0


TOLERANCE = 10e-6


def has_converged(voltage_list):
    for k in pq_buses:
        cur = voltage_list[k][-1]
        prev = (
            voltage_list[k][-2] if len(voltage_list[k]) > 1 else (0, 0)
        )  # dummy value if starting out

        if np.abs(cur[0] - prev[0]) > TOLERANCE:
            return False

    return True


while not has_converged(V) and i < 1000:
    i += 1
    # new_v = [(0, 0), (0, 0)]  # initialize new voltages with dummy values
    for k in pq_buses:
        sum_term = 0
        for j in all_buses:
            if k != j:
                # slack bus when j == 2
                V_j_polar = ((1, 0)) if j == 2 else V[j][-1]
                V_j = P2R(*V_j_polar)
                sum_term += Y[k, j] * V_j

        new_v_rect = (1 / Y[k, k]) * ((np.conjugate(S[k]) / P2R(*V[k][-1])) - sum_term)
        new_v_polar = R2P(new_v_rect)
        V[k].append(new_v_polar)
        # print("down here", k, new_v[k])


print(f"took {i} iterations")
display(V[0][-1], V[1][-1])

took 7 iterations


(0.9976294692328347, -0.01393812329009965)

(0.9983280454368093, -0.014961074673454616)